In [5]:
print("ok")

ok


In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain_pinecone import PineconeVectorStore
from sentence_transformers import SentenceTransformer

import pinecone
from pinecone import Pinecone, ServerlessSpec

from dotenv import load_dotenv
import os

c:\Users\Hamza\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [3]:
# Extract data from pdf
def load_pdf(data):
    loader = DirectoryLoader(
        data, 
        glob = "*.pdf",
        loader_cls = PyPDFLoader
    )
    documents = loader.load()
    return documents

In [4]:
extracted_data = load_pdf("data/")

In [5]:
#extracted_data

In [6]:
# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500, 
        chunk_overlap = 20
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("Length of my chunk:", len(text_chunks))

Length of my chunk: 7020


In [8]:
# download embedding model
def download_hugging_face_embedding_model():
    embeddings = HuggingFaceBgeEmbeddings(
        model_name = "sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings

In [9]:
embeddings = download_hugging_face_embedding_model()
embeddings

HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='', show_progress=False)

In [10]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [11]:
print(query_result)

[-0.010300836525857449, 0.1830792874097824, 0.030811207368969917, 0.004452838096767664, -0.027336152270436287, -0.03356252610683441, 0.037631504237651825, -0.03157331794500351, -0.003390994854271412, -0.008950844407081604, 0.03803613409399986, -0.051291029900312424, 0.00036832180921919644, -0.02372707426548004, 0.0927102267742157, -0.027795836329460144, -0.03515258803963661, -0.003224171930924058, -0.07681781053543091, -0.05761216953396797, 0.07257597893476486, 0.11128545552492142, 0.016058532521128654, 0.01590849459171295, -0.08232702314853668, 0.0070072864182293415, 0.02901313453912735, 0.0011386657133698463, 0.11671734601259232, -0.03232727572321892, -0.032271608710289, -0.0012589985271915793, 0.10591629147529602, 0.023600837215781212, 0.009664943441748619, 0.09834079444408417, 0.04293632134795189, -0.01954762451350689, 0.019267864525318146, -0.06417106837034225, 0.0239233560860157, -0.05288010463118553, -0.02646954171359539, 0.005548680666834116, -0.017025195062160492, -0.030232716

In [12]:
# Initialize Pinecone client
pinecone = Pinecone(
    api_key = PINECONE_API_KEY,
    serverless = ServerlessSpec(
        cloud = 'aws',
        region = 'us-east-1'
    )
)

# Specify your index
index_name = "medical-chatbot"

# Creating Embeddings for Each of The Text Chunks & storing
doc_search = PineconeVectorStore.from_texts(
    [t.page_content for t in text_chunks], 
    embeddings, 
    index_name = index_name
)

In [13]:
#If we already have an index we can load it like this
doc_search = PineconeVectorStore.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs = doc_search.similarity_search(query, k=3)

print("Result", docs)

Result [Document(id='6e9f7130-17a6-4e8b-ac66-8dfcc7b05aa7', metadata={}, page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly cause aller-\ngic reactions:\n• penicillin or other antibiotics'), Document(id='e675f47e-a9f8-49a2-9491-03447194cf35', metadata={}, page_content='When thisoccurs, an allergy develops against the offending sub-stance (an allergen.)'), Document(id='df9bfb6a-c6a0-4d6b-b5cb-8c6557d39ef8', metadata={}, page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance th

In [14]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [15]:
PROMPT = PromptTemplate(
    template = prompt_template, 
    input_variables = ["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

In [16]:
llm = CTransformers(
        model = "model/llama-2-7b-chat.ggmlv3.q4_0.bin",
        model_type = "llama",
        config = {
                 'max_new_tokens' : 512,
                 'temperature' : 0.8
                }
        )

In [17]:
qa = RetrievalQA.from_chain_type(
    llm = llm, 
    chain_type = "stuff", 
    retriever = doc_search.as_retriever(
        search_kwargs = {'k': 2}
    ),
    return_source_documents = True, 
    chain_type_kwargs = chain_type_kwargs
)

In [18]:
while True:
    user_input = input(f"Input Prompt:")
    result = qa.invoke({"query": user_input})
    print("Response : ", result["result"])